In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import csv
import json

In [2]:
user_train_path = '../fashion_data/processed_data/user_train.json'
user_valid_path = '../fashion_data/processed_data/user_valid.json'
user_test_path = '../fashion_data/processed_data/user_test.json'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)

In [4]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[11, 12, 13, 14, 15]
[12, 13, 14, 15, 16]
[13, 14, 15, 16, 17]


In [5]:
len(user_train_dict[user_id])

5

In [6]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


### Creating the SASRec CTR Data

In [21]:
sasrec_train_dict = {"user": [],"history_item_id": [],"history_rating": [],"item_id": [],"rating": [], "timestamp": []}
sasrec_valid_dict = {"user": [],"history_item_id": [],"history_rating": [],"item_id": [],"rating": [], "timestamp": []}
sasrec_test_dict = {"user": [],"history_item_id": [],"history_rating": [],"item_id": [],"rating": [], "timestamp": []}
user_list = list(user_train_dict.keys())

cnt =  0
for user in user_train_dict:
    # cnt += 1
    # print(f"User: {user}")
    
    # sasrec_valid_dict['user'].append(user)
    # sasrec_test_dict['user'].append(user)
    user_int = int(user)
    user_rating_dict = ratings_df[ratings_df['user'] == user_int]
    # print(f"user_rating_dict: {user_rating_dict}")

    # print(f"All train_valid_test_movies: {train_valid_test_movies}")

    # train_valid_test_ratings = []
    # for movie in train_valid_test_movies:
    #     movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
    #     if movie_rating > 3:
    #         train_valid_test_ratings.append(1)
    #     else:
    #         train_valid_test_ratings.append(0)
    # print(f"All train_valid_test_ratings: {train_valid_test_ratings}")

    ### SASRec Train Data
    # print("SASRec Train Data")
    # print(user_train_dict[user])
    sasrec_train_dict['user'].append(user_int)
    sasrec_train_dict['history_item_id'].append(user_train_dict[user][:-1])
    user_train_ratings = []
    for item in user_train_dict[user]:
        item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if item_rating > 3:
            user_train_ratings.append(1)
        else:
            user_train_ratings.append(0)
    sasrec_train_dict['history_rating'].append(user_train_ratings[:-1])
    sasrec_train_dict['item_id'].append(user_train_dict[user][-1])
    sasrec_train_dict['rating'].append(user_train_ratings[-1])
    sasrec_train_dict['timestamp'].append(0)
    # print("sasrec_train_dict:", sasrec_train_dict)

    ### SASRec Valid Data
    # print("SASRec Valid Data")
    # print(user_valid_dict[user])
    sasrec_valid_dict['user'].append(user_int)
    sasrec_valid_dict['history_item_id'].append(user_valid_dict[user][:-1])
    user_valid_ratings = []
    for item in user_valid_dict[user]:
        item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if item_rating > 3:
            user_valid_ratings.append(1)
        else:
            user_valid_ratings.append(0)
    sasrec_valid_dict['history_rating'].append(user_valid_ratings[:-1])
    sasrec_valid_dict['item_id'].append(user_valid_dict[user][-1])
    sasrec_valid_dict['rating'].append(user_valid_ratings[-1])
    sasrec_valid_dict['timestamp'].append(0)
    # print("sasrec_valid_dict:", sasrec_valid_dict)

    ### SASRec Test Data
    # print("SASRec Test Data")
    # print(user_test_dict[user])
    sasrec_test_dict['user'].append(user_int)
    sasrec_test_dict['history_item_id'].append(user_test_dict[user][:-1])
    user_test_ratings = []
    for item in user_test_dict[user]:
        # print(item)
        item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if item_rating > 3:
            user_test_ratings.append(1)
        else:
            user_test_ratings.append(0)
    sasrec_test_dict['history_rating'].append(user_test_ratings[:-1])
    sasrec_test_dict['item_id'].append(user_test_dict[user][-1])
    sasrec_test_dict['rating'].append(user_test_ratings[-1])
    sasrec_test_dict['timestamp'].append(0)
    # print("sasrec_test_dict:", sasrec_test_dict)
    # if cnt == 10:
    # break

In [22]:
print(f"""sasrec_train_dict: {len(sasrec_train_dict['user'])}\n
    sasrec_valid_dict: {len(sasrec_valid_dict['user'])}\n
    sasrec_test_dict: {len(sasrec_test_dict['user'])}\n
    """)

sasrec_train_dict: 1273

    sasrec_valid_dict: 1273

    sasrec_test_dict: 1273

    


### Saving files

In [23]:
def save_to_csv(path, data_dict):
    with open(path, 'w+', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data_dict.keys())
        # Write header
        writer.writeheader()
        # Write data rows
        for i in range(len(data_dict['user'])):
            row = {key: data_dict[key][i] for key in data_dict.keys()}
            writer.writerow(row)
    print("CSV file created successfully.")

In [25]:
train_data_path = './final_data/fashion/train.csv'
valid_data_path = './final_data/fashion/valid.csv'
test_data_path = './final_data/fashion/test.csv'

save_to_csv(train_data_path, sasrec_train_dict)
save_to_csv(valid_data_path, sasrec_valid_dict)
save_to_csv(test_data_path, sasrec_test_dict)

CSV file created successfully.
CSV file created successfully.
CSV file created successfully.
